In [14]:
import mmcv
from mmcv import Config
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.models import build_detector
from mmdet.apis import single_gpu_test
from mmcv.runner import load_checkpoint
import os
from mmcv.parallel import MMDataParallel
import pandas as pd
from pandas import DataFrame
from pycocotools.coco import COCO
import numpy as np

In [20]:
# config file 들고오기
cfg = Config.fromfile('./configs/custom_augmentation/faster_rcnn_mosaic.py')

# Multi scale
cfg.data.test.pipeline[1]['img_scale'] = [(1000, 600),(1333, 800),(1666, 1000)]

# checkpoint path
epoch = 24
checkpoint_path = os.path.join(cfg.work_dir, f'epoch_{epoch}.pth')

print(f'Config:\n{cfg.pretty_text}')

Config:
model = dict(
    type='FasterRCNN',
    pretrained='torchvision://resnet50',
    backbone=dict(
        type='ResNet',
        depth=50,
        num_stages=4,
        out_indices=(0, 1, 2, 3),
        frozen_stages=1,
        norm_cfg=dict(type='BN', requires_grad=True),
        norm_eval=True,
        style='pytorch'),
    neck=dict(
        type='FPN',
        in_channels=[256, 512, 1024, 2048],
        out_channels=256,
        num_outs=5),
    rpn_head=dict(
        type='RPNHead',
        in_channels=256,
        feat_channels=256,
        anchor_generator=dict(
            type='AnchorGenerator',
            scales=[8],
            ratios=[0.5, 1.0, 2.0],
            strides=[4, 8, 16, 32, 64]),
        bbox_coder=dict(
            type='DeltaXYWHBBoxCoder',
            target_means=[0.0, 0.0, 0.0, 0.0],
            target_stds=[1.0, 1.0, 1.0, 1.0]),
        loss_cls=dict(
            type='CrossEntropyLoss', use_sigmoid=True, loss_weight=1.0),
        loss_bbox=dict(typ

In [21]:
dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=1,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


In [22]:
model = build_detector(cfg.model, test_cfg=cfg.get('test_cfg'))
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu')

model.CLASSES = dataset.CLASSES
model = MMDataParallel(model.cuda(), device_ids=[0])

2021-05-19 05:59:59,974 - mmdet - INFO - load model from: torchvision://resnet50
2021-05-19 05:59:59,974 - mmdet - INFO - Use load_from_torchvision loader
2021-05-19 06:00:00,183 - mmdet - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias



Use load_from_local loader


In [23]:
output = single_gpu_test(model, data_loader, show_score_thr=0.05)

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 837/837, 8.1 task/s, elapsed: 104s, ETA:     0s

In [6]:
# using for ensemble
# mmcv.dump(output, "./work_dirs/faster_rcnn_2x_batch8_mosaic/epoch_24.pkl")

In [24]:
prediction_strings = []
file_names = []
coco = COCO(cfg.data.test.ann_file)
imag_ids = coco.getImgIds()

class_num = 11
for i, out in enumerate(output):
    prediction_string = ''
    image_info = coco.loadImgs(coco.getImgIds(imgIds=i))[0]
    for j in range(class_num):
        for o in out[j]:
            prediction_string += str(j) + ' ' + str(o[4]) + ' ' + str(o[0]) + ' ' + str(o[1]) + ' ' + str(
                o[2]) + ' ' + str(o[3]) + ' '
        
    prediction_strings.append(prediction_string)
    file_names.append(image_info['file_name'])


submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv(os.path.join(cfg.work_dir, f'submission_{epoch}_faster_rcnn_2x_batch8_wontrans_s50c65_num3.csv'), index=None)
submission.head()

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


,PredictionString,image_id
0,1 0.55311984 159.95706 67.323784 254.4163 166....,batch_01_vt/0021.jpg
1,1 0.12916517 0.0 203.5789 309.11206 459.68085 ...,batch_01_vt/0028.jpg
2,1 0.8322698 72.10403 174.52551 373.06055 379.2...,batch_01_vt/0031.jpg
3,1 0.11362501 363.9692 213.3289 379.76242 258.7...,batch_01_vt/0032.jpg
4,1 0.54592764 389.94 271.8695 478.0432 342.8786...,batch_01_vt/0070.jpg
